# Reading the Data

In [1]:
from Bio import SeqIO
import numpy as np
import csv
from shared import *
from project_nader import *
from time import time

In [2]:
def read_reads():
    """
    Reads all the RNA reads from reads.fa and returns a list of sequences as strings.

    :return: a list of RNA sequence reads as strings
    """
    max_id_len = 600
    reads = np.zeros((1575, 2), dtype='U{:d}'.format(max_id_len))
    i = 0
    for seq_record in SeqIO.parse("reads.fa", "fasta"):
        reads[i, 0] = seq_record.id
        reads[i, 1] = str(seq_record.seq)
        i += 1
    return reads

In [3]:
def read_genome():
    """
    Reads the genome sequence from genome.fa and return the genome sequence as a string.

    :return: a string of the genome sequence
    """
    genome = None
    for seq_record in SeqIO.parse("genome.fa", "fasta"):
        genome = str(seq_record.seq)
    return genome

In [4]:
def read_known_genes():
    """
    Reads the un/known genes, isoforms, and exons from genes.tab and constructs objects for each
    and return the list of constructed genes.

    :return: known_genes (a list of known Gene objects), unknown_genes (a list of unknown Gene objects)
    """
    known_genes, known_isoforms, known_exons = {}, {}, {}
    unknown_genes, unknown_isoforms, unknown_exons = {}, {}, {}
    with open("genes.tab") as tsv:
        for line in csv.reader(tsv, dialect="excel-tab"):
            name = line[0]
            if name == 'gene':
                known_genes[line[1]] = line[2].split(';')
            elif name == 'isoform':
                known_isoforms[line[1]] = line[2].split(';')
            elif name == 'exon':
                known_exons[line[1]] = Exon(line[1], int(line[2]), int(line[3]))
            elif name == 'unknown_gene':
                unknown_genes[line[1]] = line[2].split(';')
            elif name == 'unknown_isoform':
                unknown_isoforms[line[1]] = line[2].split(';')
            elif name == 'unknown_exon':
                unknown_exons[line[1]] = Exon(line[1], int(line[2]), int(line[3]))
    # Create the known Isoform objects
    for k in known_isoforms:
        known_isoforms[k] = Isoform(k, [known_exons[key] for key in known_isoforms[k]])
        
    # Create the UNknown Isoform objects
    for k in unknown_isoforms:
        unknown_isoforms[k] = Isoform(k, [unknown_exons[key] for key in unknown_isoforms[k]])

    # Create the known Genes objects
    for k in known_genes:
        known_genes[k] = Gene(k, [known_isoforms[key] for key in known_genes[k]])
        
    # Create the UNknown Genes objects
    for k in unknown_genes:
        unknown_genes[k] = Gene(k, [unknown_isoforms[key] for key in unknown_genes[k]])
    return known_genes, unknown_genes

# Alignment Algorithm Class

In [5]:
MIN_INTRON_SIZE = 20
MAX_INTRON_SIZE = 10000


class Aligner:
    def __init__(self, genome_sequence, known_genes):
        """
        Initializes the aligner. Do all time intensive set up here. i.e. build suffix array.

        genome_sequence: a string (NOT TERMINATED BY '$') representing the bases of the of the genome
        known_genes: a python set of Gene objects (see shared.py) that represent known genes. You can get the isoforms
                     and exons from a Gene object

        Time limit: 500 seconds maximum on the provided data. Note that our server is probably faster than your machine,
                    so don't stress if you are close. Server is 1.25 times faster than the i7 CPU on my computer
        """
        self.sa = get_suffix_array(genome_sequence)
        L = get_bwt(genome_sequence, self.sa)
        self.occ, self.M = get_occ(L), get_M(get_F(L))
        self.known_genes = known_genes

    def align(self, read_sequence):
        """
        Returns an alignment to the genome sequence. An alignment is a list of pieces.
        Each piece consists of a start index in the read, a start index in the genome, and a length
        indicating how many bases are aligned in this piece. Note that mismatches are count as "aligned".

        Note that <read_start_2> >= <read_start_1> + <length_1>. If your algorithm produces an alignment that
        violates this, we will remove pieces from your alignment arbitrarily until consecutive pieces
        satisfy <read_start_2> >= <read_start_1> + <length_1>

        Return value must be in the form (also see the project pdf):
        [(<read_start_1>, <reference_start_1, length_1), (<read_start_2>, <reference_start_2, length_2), ...]

        If no good matches are found: return the best match you can find or return []

        Time limit: 0.5 seconds per read on average on the provided data.
        """
        # for gene in self.known_genes:

        pass

# Testing

## Initializations

In [110]:
from project_tiffany import exact_suffix_matches as em
from project_tiffany import get_suffix_array as gs
from project_tiffany import get_bwt as gbwt
from project_tiffany import get_F as gF
from project_tiffany import get_M as gM
from project_tiffany import get_occ as gocc

### Reading Data

In [115]:
# Main
reads = read_reads()
genome_sequence = read_genome()
known_genes, unknown_genes = read_known_genes()

In [117]:
reverse_genome_sequence = genome_sequence[::-1]

### Initializing using project_nader

In [128]:
print("        length of the genome sequence: " + str(len(genome_sequence) + 1))

t = -time()
"""
aligner = Aligner(genome_sequence, known_genes)
t += time()
"""
# Initialization
reverse_sa = get_suffix_array(reverse_genome_sequence + '$')
reverse_L = get_bwt(reverse_genome_sequence, reverse_sa)
reverse_occ, reverse_M = get_occ(reverse_L), get_M(get_F(reverse_L))
# self.known_genes = known_genes

print("        time to run Aligner.__init__: " + str(t + time()))

        length of the genome sequence: 10949001
        time to run Aligner.__init__: 475.045284986496


In [129]:
print("        length of the genome sequence: " + str(len(genome_sequence) + 1))

t = -time()
"""
aligner = Aligner(genome_sequence, known_genes)
t += time()
"""
# Initialization
sa = get_suffix_array(genome_sequence + '$')
L = get_bwt(genome_sequence, sa)
occ, M = get_occ(L), get_M(get_F(L))
# self.known_genes = known_genes

print("        time to run Aligner.__init__: " + str(t + time()))

        length of the genome sequence: 10949001
        time to run Aligner.__init__: 443.16036200523376


### Initializing using project_tiffany

In [111]:
t = -time()
"""
aligner = Aligner(genome_sequence, known_genes)
t += time()
"""
# Initialization
sa_T = gs(reverse_genome_sequence + '$')
L_T = gbwt(reverse_genome_sequence, sa_T)
occ_T, M_T = gocc(L_T), gM(gF(L_T))
# self.known_genes = known_genes

print("        time to run Aligner.__init__: " + str(t + time()))

        time to run Aligner.__init__: 78.91130089759827


In [8]:
t = -time()
known_transcriptome = {}
for gene in known_genes:
    known_transcriptome[gene] = {}
    for iso in known_genes[gene].isoforms:
        isoform, exons, total_len = '', [], 0
        for ex in iso.exons:
            isoform += genome_sequence[ex.start: ex.end]
            exons.append((total_len, ex.start, ex.end - ex.start))
            total_len += ex.end - ex.start
        known_transcriptome[gene][iso] = [isoform, exons]
print("time to find the known transcriptome: " + str(t + time()))

time to find the known transcriptome: 0.9047422409057617


In [9]:
t = -time()
unknown_transcriptome = {}
for gene in unknown_genes:
    unknown_transcriptome[gene] = {}
    for iso in unknown_genes[gene].isoforms:
        isoform, exons, total_len = '', [], 0
        for ex in iso.exons:
            isoform += genome_sequence[ex.start: ex.end]
            exons.append((total_len, ex.start, ex.end - ex.start))
            total_len += ex.end - ex.start
        unknown_transcriptome[gene][iso] = [isoform, exons]
print("time to find the UNknown transcriptome: " + str(t + time()))

time to find the UNknown transcriptome: 0.0004279613494873047


In [10]:
num_isoforms = 0
for gene in known_transcriptome:
    num_isoforms += len(known_transcriptome[gene])
print('number of known isoforms:', num_isoforms)

number of known isoforms: 127


In [11]:
num_isoforms = 0
for gene in unknown_transcriptome:
    num_isoforms += len(unknown_transcriptome[gene])
print('number of UNknown isoforms:', num_isoforms)

number of UNknown isoforms: 9


## Alignment to the Transcriptome

In [12]:
def find_alignment(read_len, align_start, exons):
    def find_start_location(lo, hi):
        mid = (lo + hi) // 2
        if exons[mid][0] <= align_start < exons[mid][0] + exons[mid][2]:
            return mid
        elif exons[mid][0] + exons[mid][2] <= align_start:
            return find_start_location(mid+1, hi)
        else:
            return find_start_location(lo, mid-1)
    idx = find_start_location(0, len(exons))
    algn = [(0, exons[idx][1] + (align_start - exons[idx][0]),
             read_len if read_len + (align_start - exons[idx][0]) <= exons[idx][2]
             else exons[idx][2] - (align_start - exons[idx][0]))]
    read_len -= exons[idx][2] - (align_start - exons[idx][0])
    while read_len > 0:
        idx += 1
        algn.append((algn[-1][0] + algn[-1][2], exons[idx][1],
                     read_len if read_len <= exons[idx][2]
                     else exons[idx][2]))
        read_len -= exons[idx][2]
    return algn

In [13]:
def align_to_isoform(read, isoform, exons):
    match = (-1, float("inf"))
    for i in range(len(isoform) - len(read) + 1):
        j, mismatches = 0, 0
        while j < len(read):
            if isoform[i+j] != read[j]:
                mismatches += 1
            if mismatches > MAX_NUM_MISMATCHES:
                break
            j += 1
        if j == len(read) and mismatches < match[1]:
            match = (i, mismatches)
    return match if match[0] == -1 else (find_alignment(len(read), match[0], exons), match[1])

In [14]:
def align_to_transcriptome(read, transcriptome):
    match = (-1, float("inf"))
    for gene in transcriptome:
        for iso in transcriptome[gene]:
            alignment, mismatches = align_to_isoform(read, transcriptome[gene][iso][0], transcriptome[gene][iso][1])
            if alignment != -1 and mismatches <= match[1]:
                match = (alignment, mismatches)
    return match

In [15]:
reads[0][1]

'ATTACTCTTGGGAATGAAATCCTATCTATATAAGCTGTGGTTTGAAATCC'

In [16]:
t = -time()
# align_to_isoform(reads[166][1], reads[166][1])
align_to_transcriptome(reads[150][1], known_transcriptome)

([(0, 6759550, 50)], 2)

In [18]:
unable_to_match = []
avg_t, num_matches = 0, 0
for i in range(len(reads)):
    t = -time()
    start, mismatches = align_to_transcriptome(reads[i][1], known_transcriptome)
    if start == -1:
        unable_to_match.append(i)
        print("      " + str(i) + ' not matched!')
    else:
        tt = time() + t
#         print("      " + str(tt))
        avg_t += tt
        num_matches += 1

      2 not matched!
      3 not matched!
      8 not matched!
      29 not matched!
      61 not matched!
      84 not matched!
      91 not matched!
      104 not matched!
      135 not matched!
      154 not matched!
      163 not matched!
      167 not matched!
      173 not matched!
      179 not matched!
      200 not matched!
      201 not matched!
      213 not matched!
      218 not matched!
      253 not matched!
      268 not matched!
      271 not matched!
      276 not matched!
      281 not matched!
      293 not matched!
      300 not matched!
      322 not matched!
      323 not matched!
      358 not matched!
      390 not matched!
      392 not matched!
      393 not matched!
      413 not matched!
      415 not matched!
      430 not matched!
      438 not matched!
      444 not matched!
      451 not matched!
      454 not matched!
      467 not matched!
      475 not matched!
      484 not matched!
      499 not matched!
      501 not matched!
      506 not matched

In [19]:
print('    Average time: ' + str(avg_t / num_matches))

    Average time: 0.43652803950987157


In [61]:
still_unable_to_match = []
for i in unable_to_match:
    alignment, mismatches = align_to_transcriptome(reads[i][1], unknown_transcriptome)
    if alignment != -1:
        print('          ' + str(i) + ': ' + str(alignment) + ", " + str(mismatches))
    else:
        still_unable_to_match.append(i)

          8: [(0, 6496171, 22), (22, 6514428, 28)], 2
          84: [(0, 6500926, 27), (27, 6514428, 23)], 3
          91: [(0, 2590477, 50)], 3
          104: [(0, 2559519, 50)], 2
          167: [(0, 6455646, 50)], 5
          173: [(0, 3920513, 50)], 2
          213: [(0, 6462138, 34), (34, 6496027, 16)], 2
          218: [(0, 6496167, 26), (26, 6500749, 24)], 4
          253: [(0, 3941290, 50)], 5
          271: [(0, 6455465, 50)], 4
          276: [(0, 6455594, 50)], 0
          322: [(0, 2573320, 50)], 4
          323: [(0, 3941393, 50)], 1
          390: [(0, 2574410, 50)], 3
          392: [(0, 6514438, 50)], 2
          415: [(0, 6461410, 50)], 2
          430: [(0, 3941311, 50)], 5
          499: [(0, 2564134, 50)], 1
          510: [(0, 6496086, 50)], 4
          528: [(0, 6455479, 50)], 1
          568: [(0, 2580032, 50)], 5
          625: [(0, 3941385, 50)], 2
          657: [(0, 2588594, 44), (44, 2590345, 6)], 4
          707: [(0, 6455505, 50)], 3
          729: [(0, 39

In [26]:
print('          ' + str(len(still_unable_to_match)))

          75


## Alignment to the Genome

In [95]:
from itertools import product

In [28]:
def replace_base_at_index(read_sequence, idx, base):
    length = len(read_sequence)
    return read_sequence[:idx] + base + read_sequence[idx + 1:]

In [29]:
BASES

['A', 'C', 'G', 'T']

In [50]:
def find_seeds(read_sequence, mismatches=6, k=3):
    read_sequence = read_sequence[::-1]
    len_read, seeds_len, seeds = len(read_sequence), 0, []
    if not k:
        return []
    else:
        _range, length = exact_suffix_matches(read_sequence, M, occ)
        max_so_far, updated = (_range, length, read_sequence, 0), True
        
        while max_so_far[3] < mismatches and max_so_far[1] < len_read and updated:
            updated = False
            possible_changes = [b for b in BASES if b != max_so_far[2][len_read - max_so_far[1] - 1]]
            misses = max_so_far[3]
            for change in possible_changes:
                new_read = replace_base_at_index(read_sequence, len_read - length - 1, change)
                new_range, new_length = exact_suffix_matches(new_read, M, occ)
                if new_length > max_so_far[1]:
                    max_so_far = (new_range, new_length, new_read, misses + 1)
                    updated = True
            
        seeds = [([sa[i] for i in max_so_far[0]], max_so_far[1], max_so_far[3], max_so_far[2][len_read - max_so_far[1]:])]
        if len_read == max_so_far[1]:
            return seeds
        else:
            return seeds + find_seeds(max_so_far[2][:len(max_so_far[2]) - max_so_far[1]],
                                      mismatches=mismatches - max_so_far[3], k=k-1)

In [82]:
print('          ' + reads[8][1])
print('          ' + genome_sequence[6496171:6496171+22] + "&" +  genome_sequence[6514428:6514428+28])

          TAATCAGAAGGTGGATCAACTGGAAGATGTGCCTCCTCCAAAGAGCCGTA
          TAATCAGAAGGTGGATCAAGTG&GAAGATGTGCCACCTCCAAAGAGCCGTA


In [125]:
reverse_read = reads[8][1][::-1]
_range, length = exact_suffix_matches(reverse_read, M, occ)

In [126]:
print('          ' + reverse_read[-length:])
print('          ' + reverse_genome_sequence[sa[_range[0]]:sa[_range[0]]+length])

          TGGAAGACTAAT
          TGGAAGACTAAG


In [127]:
a = reverse_genome_sequence[len(reverse_genome_sequence) - 6496171 - 19:len(reverse_genome_sequence) - 6496171]
a
print('          ' + reverse_read)
print('          ' + reverse_genome_sequence[len(reverse_genome_sequence) - 6514428 - 28:len(reverse_genome_sequence) - 6514428]
      + "&" +  reverse_genome_sequence[len(reverse_genome_sequence) - 6496171 - 22:len(reverse_genome_sequence) - 6496171])

          ATGCCGAGAAACCTCCTCCGTGTAGAAGGTCAACTAGGTGGAAGACTAAT
          ATGCCGAGAAACCTCCACCGTGTAGAAG&GTGAACTAGGTGGAAGACTAAT


In [114]:
em(b, M_T, occ_T)

((2366925, 2366926), 15)

In [91]:
b[-12:]

'TGGAAGACTAAT'

In [64]:
print('          ' + reads[8][1][::-1])
find_seeds(reads[8][1])

          ATGCCGAGAAACCTCCTCCGTGTAGAAGGTCAACTAGGTGGAAGACTAAT


[([8992522, 2058582], 13, 1, 'CTGGAAGACTAAT'),
 ([10949000, 8353711], 1, 0, 'A'),
 ([4908239, 4622709], 15, 1, 'AAGAAGGTCAACTAG')]

In [ ]:
for i in unable_to_match:
    seeds = find_seeds(reads[i][1])
    p

In [ ]:
t = -time()
read_sequence = reads[0][1].copy()
rnge1, length1 = exact_suffix_matches(read_sequence, M, occ)
print('       ', length1)
lens = {}
for base in ALPHABET:
    read_sequence = replace_base_at_index(read_sequence, length1, base)
    lens[base] = exact_suffix_matches(read_sequence, M, occ)
    print('        ', base, lens[base])

best_letter = max(lens, key=lambda x: lens.get(x)[1])
length1 = lens[best_letter][1]
for base in ALPHABET:
    read_sequence = replace_base_at_index(read_sequence, length1, base)
    lens[base] = exact_suffix_matches(read_sequence, M, occ)
    print('        ', base, lens[base])

best_letter = max(lens, key=lambda x: lens.get(x)[1])

In [ ]:
print('         ' + str([sa[i] for i in rnge1]) + " " + str(length1))
rnge2, length2 = exact_suffix_matches(reads[0][1][:len(reads[0][1]) - length1 - 0], M, occ)
print('         ' + str([sa[i] for i in rnge2]) + " " + str(length2))
rnge3, length3 = exact_suffix_matches(reads[0][1][:len(reads[0][1]) - length1 - length2 - 0], M, occ)
print('         ' + str([sa[i] for i in rnge3]) + " " + str(length))
print("time to find the read in the genome: " + str(t + time()))


In [ ]:
"""
Use exact_suffix_matches but need to find consecutive matches with possible mismatches and restricted intron length
"""
def align_to_genome(read):
    # TODO
    pass

# Evaluation

In [ ]:
from evaluation import *

In [ ]:
known_isoforms = []
for gene in known_genes:
    known_isoforms.extend(known_genes[gene].isoforms)

In [ ]:
unknown_isoforms = []
for gene in unknown_genes:
    unknown_isoforms.extend(unknown_genes[gene].isoforms)

In [ ]:
genome_isoform_offsets = index_isoform_locations(known_isoforms, unknown_isoforms)

In [ ]:
read_sequence = reads[150][1]
alignment = align_to_transcriptome(reads[150][1], known_transcriptome)[0]
alignment

In [ ]:
CASE_GENE

In [ ]:
unable_to_match = []
for i in range(len(reads)):
    read_sequence = reads[i][1]
    alignment, mismatches = align_to_transcriptome(read_sequence, known_transcriptome)
    if alignment == -1:
        unable_to_match.append(i)
        print('      ', str(i) + ' not matched!')
    else:
        case, _ = evaluate_alignment(genome_sequence, read_sequence, alignment, unknown_isoforms, genome_isoform_offsets)
        if case != CASE_GENE:
            print('      ', i)
            print('      ', case, 'alignment')
            print('      ', alignment, read_sequence)

In [ ]:
print('size of unmatched: ', len(unable_to_match))
print('unmatched read indices:', unable_to_match)